In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def generate(name):
    num_sessions = 0
    inputs = []
    outputs = []
    with open(name, 'r') as f:
        for line in tqdm(f,"loading data"):
            num_sessions += 1
            line = tuple(map(lambda n: n - 1, map(int, line.strip().split())))
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])
    print('Number of sessions({}): {}'.format(name, num_sessions))
    print('Number of seqs({}): {}'.format(name, len(inputs)))
    dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
    return dataset


class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_keys):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_0 = nn.LSTMCell(input_size, hidden_size)
        self.cell_1 = nn.LSTMCell(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_keys)

    def forward(self, x):
        h0 = torch.zeros(x.size(1), self.hidden_size).to(device)
        c0 = torch.zeros(x.size(1), self.hidden_size).to(device)
        h1 = torch.zeros(x.size(1), self.hidden_size).to(device)
        c1 = torch.zeros(x.size(1), self.hidden_size).to(device)
        for xt in x:
            h0, c0 = self.cell_0(xt, (h0, c0))  # l0层直接接受xt输入
            h1, c1 = self.cell_1(h0, (h1, c1))  # l1层接受l0层的输出h为输入

        out = self.fc(h1)
        return out

In [3]:
# Hyperparameters
num_classes = 28
num_epochs = 300
batch_size = 1024
input_size = 1
model_dir = 'model'
log = 'Lstm_Cell_Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
num_layers = 2
hidden_size = 64
window_size = 10
file_dir = 'data/'

In [9]:
seq_dataset = generate(file_dir+'hdfs_train')
dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

loading data: 4855it [00:00, 22531.56it/s]

Number of sessions(data/hdfs_train): 4855
Number of seqs(data/hdfs_train): 46575


In [4]:
model = Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [5]:
model.load_state_dict(torch.load(model_dir + '/' + log + '.pt'))

<All keys matched successfully>

In [10]:
writer = SummaryWriter(log_dir='log/' + log)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# Train the model
total_step = len(dataloader)
current_epoch = 0

In [11]:
num_epochs = 100

In [12]:
start_time = time.time()

for epoch in range(current_epoch,current_epoch+num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    for step, (seq, label) in enumerate(dataloader):
        # Forward pass
        seq = seq.clone().detach().view(-1, window_size, input_size).permute(1,0,2).to(device)
        output = model(seq)
        loss = criterion(output, label.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        writer.add_graph(model, seq)
    current_epoch += 1
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
    writer.add_scalar('train_loss', train_loss / total_step, epoch + 1)
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))

E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\tensor.py:461: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  'incorrect results).', category=RuntimeWarning)


Epoch [1/100], train_loss: 0.2195
KeyboardInterrupt: 

At:
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\jit\__init__.py(300): _get_interpreter_name_for_var
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\rnn.py(944): forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(516): _slow_forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(530): __call__
  <ipython-input-2-6ad89cfa314f>(33): forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(516): _slow_forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(530): __call__
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\jit\__init__.py(1034): trace_module
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\jit\__init__.py(882): trace
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\utils\te

RuntimeError: KeyboardInterrupt: 

At:
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\jit\__init__.py(300): _get_interpreter_name_for_var
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\rnn.py(944): forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(516): _slow_forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(530): __call__
  <ipython-input-2-6ad89cfa314f>(33): forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(516): _slow_forward
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\nn\modules\module.py(530): __call__
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\jit\__init__.py(1034): trace_module
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\jit\__init__.py(882): trace
  E:\Programs\Anaconda\envs\cuda_pytorch\lib\site-packages\torch\utils\tensorboard\_pytorch_g

In [14]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + log + '.pt')
writer.close()
print('Finished Training')

Finished Training


In [7]:
def generate_test_data(name):
    hdfs = set()
    # hdfs = []
    with open('data/' + name, 'r') as f:
        for ln in f.readlines():
            ln = list(map(lambda n: n - 1, map(int, ln.strip().split())))
            ln = ln + [-1] * (window_size + 1 - len(ln))
            hdfs.add(tuple(ln))
            # hdfs.append(tuple(ln))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs

In [8]:
model.eval()
# print('model_path: {}'.format(model_path))
test_normal_loader = generate_test_data('hdfs_test_normal')
test_abnormal_loader = generate_test_data('hdfs_test_abnormal')

Number of sessions(hdfs_test_normal): 14177
Number of sessions(hdfs_test_abnormal): 4123


In [13]:
TP = 0
FP = 0
num_candidates = 5
# Test the model
start_time = time.time()
with torch.no_grad():
    for line in tqdm(test_normal_loader,"normal"):
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            label = line[i + window_size]
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).permute(1,0,2).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if label not in predicted:
                FP += 1
                break
with torch.no_grad():
    for line in tqdm(test_abnormal_loader,"abnormal"):
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            label = line[i + window_size]
            seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).permute(1,0,2).to(device)
            label = torch.tensor(label).view(-1).to(device)
            output = model(seq)
            predicted = torch.argsort(output, 1)[0][-num_candidates:]
            if label not in predicted:
                TP += 1
                break
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
# Compute precision, recall and F1-measure
FN = len(test_abnormal_loader) - TP
P = 100 * TP / (TP + FP)
R = 100 * TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
print('Finished Predicting')

abnormal: 100%|████████████████████████████████████████████████████████████████████| 4123/4123 [11:17<00:00,  6.08it/s]

elapsed_time: 4895.443s
false positive (FP): 1124, false negative (FN): 45, Precision: 78.393%, Recall: 98.909%, F1-measure: 87.464%
Finished Predicting


In [ ]:
def generate(model):
    model.eval()
    start = np.random.randint(0,len(num_classes))+1
    